In [1]:
import os
from astropy.table import Table
import time
import dictionaries
import params
import debias

data = Table.read(params.data_file)
output_directory = params.output_directory

if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

In [2]:
t_start = time.time()
for q in dictionaries.questions.keys():
    answers = dictionaries.questions[q]['answers']
    for a in answers:
        print('---------------------------------------------')
        print('Q: {}'.format(q))
        print('A: {}'.format(a))
        print('---------------------------------------------')
        data = debias.debias(data, params=params, dictionaries=dictionaries ,question=q, answer=a, verbose=True)
    break

data.write(os.path.join(output_directory, 'nsa_debiased_frist_question_only.csv'), overwrite=True)
t_end = time.time()
print('time to run: {0} hours'.format((t_end - t_start) / 3600 ))

---------------------------------------------
Q: t01_smooth_or_features
A: a01_smooth
---------------------------------------------


/Users/coleman/Desktop/zooniverse/Debiasing/sample.py:16: RuntimeWarning: divide by zero encountered in log10
  full_data['logR50'] = np.log10(full_data[params.R50_column])


Primary question, so all 297590 galaxies used.
Bin-accretion...
99  initial bins.
Reassign bad bins...
26  good bins.
Modified Lloyd algorithm...
24  iterations.
Unbinned pixels:  0  /  11559
Fractional S/N scatter (%): 14.4543781663
------------------
chisq (log) = 1818.5
chisq (exp) = 1282.2
------------------
=> exp function preferred


/Users/coleman/Desktop/zooniverse/Debiasing/fit.py:98: RuntimeWarning: invalid value encountered in log10
  term = constant*np.log10(var)
/Users/coleman/anaconda/envs/python3/lib/python3.5/site-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


--- Selected function (k): ---
exp(M),exp(R),exp(z)
--- Selected function (c): ---
linear(M),linear(R),linear(z)
fit method preferred
---------
chi2 (bin) = 0.2
chi2 (fit) = 0.03
---------
---------------------------------------------
Q: t01_smooth_or_features
A: a02_features_or_disk
---------------------------------------------


...

--- Selected function (k): ---
linear(M),linear(R),linear(z)
--- Selected function (c): ---
exp(M),exp(R),exp(z)
fit method preferred
---------
chi2 (bin) = 0.12
chi2 (fit) = 0.04
---------
---------------------------------------------
Q: t01_smooth_or_features
A: a03_star_or_artifact
---------------------------------------------


/Users/coleman/Desktop/zooniverse/Debiasing/sample.py:16: RuntimeWarning: divide by zero encountered in log10
  full_data['logR50'] = np.log10(full_data[params.R50_column])


Primary question, so all 297590 galaxies used.
Bin-accretion...
99  initial bins.
Reassign bad bins...
26  good bins.
Modified Lloyd algorithm...
24  iterations.
Unbinned pixels:  0  /  11559
Fractional S/N scatter (%): 14.4543781663
------------------
chisq (log) = 105.2
chisq (exp) = 78.7
------------------
=> exp function preferred


/Users/coleman/Desktop/zooniverse/Debiasing/fit.py:98: RuntimeWarning: invalid value encountered in log10
  term = constant*np.log10(var)


--- Selected function (k): ---
linear(M),linear(R),linear(z)
--- Selected function (c): ---
linear(M),linear(R),linear(z)
fit method preferred
---------
chi2 (bin) = 0.27
chi2 (fit) = 0.02
---------
time to run: 0.8140535869201024 hours
